In [1]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd

tf.enable_eager_execution()
tf.executing_eagerly()

True

In [2]:
import pandas as pd
df=pd.read_pickle('../data/interim/train_all_row.pickle')

In [6]:
df_val = pd.read_pickle('../data/interim/val_row.pickle')

In [4]:
df.head()

,sid,click_mode,distance_plan,eta,price,transport_mode,plan_time,pid,req_time,o_long,...,is_holiday,max_temp,min_temp,wind,dy,dyq,q,qdy,xq,xydy
0,2848914.0,1.0,53156.0,6456.0,,1.0,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,...,0,9,-2,12,0,1,0,0,0,0
1,2848914.0,1.0,48112.0,3535.0,,3.0,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,...,0,9,-2,12,0,1,0,0,0,0
2,2848914.0,1.0,48112.0,3655.0,16500,4.0,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,...,0,9,-2,12,0,1,0,0,0,0
3,2848914.0,1.0,51641.0,8871.0,1200,1.0,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,...,0,9,-2,12,0,1,0,0,0,0
4,2848913.0,2.0,10634.0,2355.0,2000,10.0,2018-11-17 22:25:16,101804.0,2018-11-17 22:25:16,116.41,...,0,9,-2,12,0,1,0,0,0,0


In [7]:
df_val.head()

,sid,click_mode,distance_plan,eta,price,transport_mode,plan_time,pid,req_time,o_long,...,is_holiday,max_temp,min_temp,wind,dy,dyq,q,qdy,xq,xydy
1258547,697184.0,1.0,7594.0,1361.0,2200,4.0,2018-11-15 05:16:06,132790.0,2018-11-15 05:16:06,116.42,...,0,8,1,34,0,0,0,0,0,1
1258549,697184.0,1.0,3932.0,3605.0,0,5.0,2018-11-15 05:16:06,132790.0,2018-11-15 05:16:06,116.42,...,0,8,1,34,0,0,0,0,0,1
272907,673286.0,2.0,9558.0,2875.0,400,2.0,2018-11-15 05:16:52,0.0,2018-11-15 05:16:52,116.37,...,0,8,1,34,0,0,0,0,0,1
272911,673286.0,2.0,8900.0,3373.0,400,1.0,2018-11-15 05:16:52,0.0,2018-11-15 05:16:52,116.37,...,0,8,1,34,0,0,0,0,0,1
272909,673286.0,2.0,7916.0,2390.0,0,6.0,2018-11-15 05:16:52,0.0,2018-11-15 05:16:52,116.37,...,0,8,1,34,0,0,0,0,0,1


In [5]:
features_sample = [
    'distance_plan',
    'eta',
    'price'
]

In [8]:
import pickle
with(open('../data/interim/features_row_sample.pickle', 'wb')) as fo:
    pickle.dump(features_sample, fo)

In [27]:
_TRAIN_DATA_PATH="../data/interim/train_sample.libsvm"
_TEST_DATA_PATH="../data/interim/val_sample.libsvm"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"
# _LOSS="sigmoid_cross_entropy_loss"

_LIST_SIZE=10

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=3

# Parameters to the scoring function.
_BATCH_SIZE=100
_HIDDEN_LAYER_DIMS=["20", "10"]

In [10]:
def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

    train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()

In [11]:
def example_feature_columns():
    """Returns the example feature columns."""
    feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
    ]
    return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
    }

def make_score_fn():
        """Returns a scoring function to build `EstimatorSpec`."""
        
        def _score_fn(context_features, group_features, mode, params, config):
            """Defines the network to score a documents."""
            del params
            del config
            # Define input layer.
            example_input = [
                tf.layers.flatten(group_features[name])
                for name in sorted(example_feature_columns())
            ]
            input_layer = tf.concat(example_input, 1)

            cur_layer = input_layer
            for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
                cur_layer = tf.layers.dense(
                  cur_layer,
                  units=layer_width,
                  activation="tanh")

            logits = tf.layers.dense(cur_layer, units=1)
            return logits
        return _score_fn

In [12]:
def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns

In [13]:
def get_estimator(hparams):
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """
    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=hparams.learning_rate,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
        params=hparams)

- Kleiner Batch bis lodd null eitf
- prüfe of f1 score 1 ist 
- sind features richtig? 
- prüfe ob mehrheitsklasse 
- vakudation loss einfügen 

In [63]:
def ltr_to_submission(df, features, ranker, path):
    features = features + ['sid', 'transport_mode']
    
    preds = ranker.predict(input_fn=lambda: input_fn(path))
    import itertools
    import numpy as np
    # Not sure how to get all preds because it runs infinit
    # So I take all till list size
    preds_slice = itertools.islice(preds, len(df)) 
    count=0
    a = np.zeros((len(df), _LIST_SIZE))

    for i in preds_slice:
        a[count]=i
        count+=1
        
        
    test_X = df[features]
    
    # Assign prediction vals to df
    # Tried with a or a sum for all features
    # a = a[:,0:_NUM_FEATURES]
    # a = a.sum(axis=1)
    
    test_X = test_X.assign(yhat = a[:,0])
    
    df_end = pd.DataFrame(columns=['yhat'], index=df.sid.unique())

    df_end = test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()[[
        'yhat', 'transport_mode'
    ]]
    
    from sklearn.metrics import f1_score
    score = f1_score(df.groupby("sid").first()['click_mode'], df_end.transport_mode, average='weighted')
    print('F1 Score is: {}'.format(score))
    
    return df_end

# Implementation

In [ ]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

In [ ]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

In [ ]:
features = pd.read_pickle("../data/interim/features_row_sample.pickle")

In [ ]:
df_train = pd.read_pickle('../data/interim/train_row.pickle')

In [ ]:
df_val = pd.read_pickle('../data/interim/val_row.pickle')

In [ ]:
df_preds = ltr_to_submission(df_val, features, ranker, _TEST_DATA_PATH)

## Step 1000, batch size 1000

In [29]:
_BATCH_SIZE=100
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpme32k2ti', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1132db1d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpme32k2ti/model.ckpt-1000


Exception ignored in: <generator object EstimatorV2.predict at 0x7fa126a56318>
Traceback (most recent call last):
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/estimator.py", line 639, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5253, in get_controller
    yield g
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5069, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framewo

Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmpme32k2ti/model.ckpt.
INFO:tensorflow:loss = 0.60961604, step = 1001
INFO:tensorflow:global_step/sec: 23.1969
INFO:tensorflow:loss = 0.6079875, step = 1101 (4.313 sec)
INFO:tensorflow:global_step/sec: 24.4329
INFO:tensorflow:loss = 0.5624323, step = 1201 (4.096 sec)
INFO:tensorflow:global_step/sec: 24.3464
INFO:tensorflow:loss = 0.5608152, step = 1301 (4.105 sec)
INFO:tensorflow:global_step/sec: 23.4339
INFO:tensorflow:loss = 0.5876624, step = 1401 (4.269 sec)
INFO:tensorflow:global_step/sec: 24.402
INFO:tensorflow:loss = 0.63809097, step = 1501 (4.099 sec)
INFO:tensorflow:global_step/sec: 24.46
INFO:tensorflow:loss = 0.5568828, step = 1601 (4.089 sec)
INFO:tensorflow:global_step/sec: 24.2375
INFO:tensorflow:loss = 0.6294006, step = 1701 (4.122 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:loss = 0.5915664, step = 8801 (3.817 sec)
INFO:tensorflow:global_step/sec: 25.6143
INFO:tensorflow:loss = 0.60111594, step = 8901 (3.903 sec)
INFO:tensorflow:global_step/sec: 25.9666
INFO:tensorflow:loss = 0.57978624, step = 9001 (3.852 sec)
INFO:tensorflow:global_step/sec: 31.2105
INFO:tensorflow:loss = 0.5691194, step = 9101 (3.207 sec)
INFO:tensorflow:global_step/sec: 28.8555
INFO:tensorflow:loss = 0.59486103, step = 9201 (3.463 sec)
INFO:tensorflow:global_step/sec: 29.1312
INFO:tensorflow:loss = 0.6040234, step = 9301 (3.433 sec)
INFO:tensorflow:global_step/sec: 28.8036
INFO:tensorflow:loss = 0.5848078, step = 9401 (3.471 sec)
INFO:tensorflow:global_step/sec: 27.2009
INFO:tensorflow:loss = 0.6277044, step = 9501 (3.677 sec)
INFO:tensorflow:global_step/sec: 31.0984
INFO:tensorflow:loss = 0.58981705, step = 9601 (3.216 sec)
INFO:tensorflow:global_step/sec: 28.3533
INFO:tensorflow:loss = 0.63769186, step = 9701 (3.528 sec)
INFO:tensorflow:global_step/sec: 27.6127
INFO:

## Target : 0 / Not relevant: 100

In [32]:
_TRAIN_DATA_PATH="../data/interim/train_sample_100.libsvm"
_TEST_DATA_PATH="../data/interim/val_sample_100.libsvm"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"
# _LOSS="sigmoid_cross_entropy_loss"

_LIST_SIZE=10

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=3

# Parameters to the scoring function.
_BATCH_SIZE=100
_HIDDEN_LAYER_DIMS=["20", "10"]

In [33]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgggobkwp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f61b617b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpgggobkwp/model.ckpt.
INFO:tensorflow:loss = 72.05671, step = 1
INFO:tensorflow:global_step/sec: 24.2699
INFO:tensorflow:loss = 62.32413, step = 101 (4.123 sec)
INFO:tensorflow:global_step/sec: 24.5237
INFO:tensorflow:loss = 58.669132, step = 201 (4.077 sec)
INFO:tensorflow:global_step/sec: 24.305
INFO:tensorflow:loss = 55.654488, step = 301 (4.114 sec)
INFO:tensorflow:global_step/sec: 23.8419
INFO:tensorflow:loss = 57.56902, step = 401 (4.197 sec)
INFO:tensorflow:global_step/sec: 23.9799
INFO:tensorflow:loss = 56.645607, step = 501 (4.168 sec)
INFO:tensorflow:global_step/sec: 24.0732
INFO:tensorflow:loss = 59.62736, step = 601 (4.157 sec)
INFO:tensorflow:global_step/sec: 24.6752
INFO:tensorflow:loss = 55.808727, step = 701 (4.049 sec)
INFO:tensorflow:global_step/sec: 23.9586
INFO:tensorflow:loss = 56.053493, step = 801 (4.177 sec)
INFO:tensorflow:glob

In [37]:
df_preds = ltr_to_submission(df_val, features, ranker, _TEST_DATA_PATH)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgggobkwp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0.31663914860828174


Doesn't change anything if I have the labels like that 

## Other loss function

In [39]:
_TRAIN_DATA_PATH="../data/interim/train_sample_100.libsvm"
_TEST_DATA_PATH="../data/interim/val_sample_100.libsvm"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
# _LOSS="pairwise_logistic_loss"
_LOSS="sigmoid_cross_entropy_loss"

_LIST_SIZE=10

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=3

# Parameters to the scoring function.
_BATCH_SIZE=100
_HIDDEN_LAYER_DIMS=["20", "10"]
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=10000)
df_preds = ltr_to_submission(df_val, features, ranker, _TEST_DATA_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpskjqix2m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9ea0033b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkpo

INFO:tensorflow:loss = -6956.0947, step = 7001 (3.867 sec)
INFO:tensorflow:global_step/sec: 25.894
INFO:tensorflow:loss = -7243.0625, step = 7101 (3.862 sec)
INFO:tensorflow:global_step/sec: 25.4483
INFO:tensorflow:loss = -7278.271, step = 7201 (3.930 sec)
INFO:tensorflow:global_step/sec: 27.0743
INFO:tensorflow:loss = -7116.8457, step = 7301 (3.693 sec)
INFO:tensorflow:global_step/sec: 26.3663
INFO:tensorflow:loss = -7292.9404, step = 7401 (3.793 sec)
INFO:tensorflow:global_step/sec: 26.0408
INFO:tensorflow:loss = -7392.0103, step = 7501 (3.840 sec)
INFO:tensorflow:global_step/sec: 25.1158
INFO:tensorflow:loss = -7372.082, step = 7601 (3.981 sec)
INFO:tensorflow:global_step/sec: 25.9324
INFO:tensorflow:loss = -7396.8564, step = 7701 (3.856 sec)
INFO:tensorflow:global_step/sec: 25.5512
INFO:tensorflow:loss = -7560.8604, step = 7801 (3.914 sec)
INFO:tensorflow:global_step/sec: 26.3208
INFO:tensorflow:loss = -7538.0493, step = 7901 (3.799 sec)
INFO:tensorflow:global_step/sec: 24.9387
INF

## Default loss function and all features

In [55]:
features = pd.read_pickle("../data/interim/features_rows_pp.pickle")

In [ ]:
_TRAIN_DATA_PATH="../data/interim/train.libsvm"
_TEST_DATA_PATH="../data/interim/val.libsvm"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"
#_LOSS="sigmoid_cross_entropy_loss"

_LIST_SIZE=100

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=90

# Parameters to the scoring function.
_BATCH_SIZE=1000
_HIDDEN_LAYER_DIMS=["20", "10"]
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=10000)
df_preds = ltr_to_submission(df_val, features, ranker, _TEST_DATA_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpthrua19r', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f65e246d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.


Exception ignored in: <generator object EstimatorV2.predict at 0x7fa111176840>
Traceback (most recent call last):
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/estimator.py", line 639, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5253, in get_controller
    yield g
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 5069, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framewo

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpthrua19r/model.ckpt.
INFO:tensorflow:loss = 68.48445, step = 1
INFO:tensorflow:global_step/sec: 0.402639
INFO:tensorflow:loss = 57.35151, step = 101 (248.366 sec)
INFO:tensorflow:global_step/sec: 0.408918
INFO:tensorflow:loss = 54.190384, step = 201 (244.548 sec)
INFO:tensorflow:Saving checkpoints for 242 into /tmp/tmpthrua19r/model.ckpt.
INFO:tensorflow:global_step/sec: 0.397634
INFO:tensorflow:loss = 52.57144, step = 301 (251.488 sec)
INFO:tensorflow:global_step/sec: 0.397856
INFO:tensorflow:loss = 54.82545, step = 401 (251.347 sec)
INFO:tensorflow:Saving checkpoints for 481 into /tmp/tmpthrua19r/model.ckpt.
INFO:tensorflow:global_step/sec: 0.397669
INFO:tensorflow:loss = 55.66167, step = 501 (251.463 sec)
INFO:tensorflow:global_st

In [64]:
df_preds = ltr_to_submission(df_val, features, ranker, _TEST_DATA_PATH)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpthrua19r/model.ckpt-7111
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
F1 Score is: 0.3135041527368519


In [ ]:
print("HI")